In [55]:
import pandas as pd
data = pd.read_csv('C:/Users/DEEPADHARSHINI/OneDrive/Desktop/Tourism/MergedTourismData.csv')

data.head()

,TransactionId,UserId,ContinentId,Continent,RegionId,Region,CountryId,Country,CityId,CityName,AttractionId,Attraction,AttractionAddress,AttractionTypeId,AttractionType,VisitYear,VisitMonth,VisitMode,VisitModeName,Rating
0,3,70456,5,Europe,21,Western Europe,163,United Kingdom,4341,Guildford,640,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,2022,10,2,Couples,5
1,9,79069,2,America,9,South America,54,Brazil,774,Brazil,640,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,2022,10,3,Family,5
2,18,43471,2,America,8,Northern America,51,United States,1806,San Francisco,640,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,2022,10,4,Friends,5
3,29,62907,2,America,8,Northern America,51,United States,50,Alexandria,640,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,2022,9,2,Couples,5
4,39,38726,4,Australia & Oceania,16,Oceania,115,New Zealand,5506,Christchurch,640,Sacred Monkey Forest Sanctuary,"Jl. Monkey Forest, Ubud 80571 Indonesia",63,Nature & Wildlife Areas,2022,9,4,Friends,3


In [ ]:
data.info()
data.shape

In [ ]:
data.describe

In [ ]:
missing_data=data.isnull().sum()
missing_data

In [ ]:
data.nunique()



In [56]:
value_counts=data['VisitModeName'].value_counts()
print(value_counts)

VisitModeName
Couples     21617
Family      15215
Friends     10944
Solo         4523
Business      623
Name: count, dtype: int64


In [ ]:
corelation_matrix=data[['TransactionId','UserId','ContinentId','RegionId','CountryId','CityId','AttractionId','AttractionTypeId','VisitYear','VisitMonth','VisitMode','Rating']].corr()
print(corelation_matrix)

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from category_encoders import TargetEncoder
from imblearn.over_sampling import SMOTE
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,classification_report
import xgboost
from xgboost import XGBClassifier


In [58]:
selected_columns=["VisitYear", "VisitMonth", "AttractionId", 
                     "ContinentId", "RegionId", "Attraction", "AttractionType", "AttractionTypeId","Rating"]

x=data[selected_columns]
y=data['VisitModeName']

categorical_columns=['AttractionType']
label_encoder = LabelEncoder()
y = pd.Series(label_encoder.fit_transform(y))

ohe=OneHotEncoder(handle_unknown='ignore',sparse_output=False)
encoded=ohe.fit_transform(x[categorical_columns])
encoded_df=pd.DataFrame(encoded,columns=ohe.get_feature_names_out(categorical_columns))

#print(encoded_df)
te=TargetEncoder()
x['Attraction']=te.fit_transform(x['Attraction'],y)

x=x.drop(columns=categorical_columns)
x=pd.concat([x,encoded_df],axis=1)
#print(x)



C:\Users\DEEPADHARSHINI\AppData\Local\Temp\ipykernel_3824\491634137.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Attraction']=te.fit_transform(x['Attraction'],y)


In [68]:
print(y)

sm=SMOTE(random_state=42)
x_smote,y_smote=sm.fit_resample(x,y)


0        1
1        2
2        3
3        1
4        3
        ..
52917    2
52918    2
52919    1
52920    2
52921    1
Length: 52922, dtype: int64


In [69]:
scaler=StandardScaler()
x_train_scaled=scaler.fit_transform(x_smote)
#x_test_scaled=scaler.transform(x_test)


In [70]:
x_train,x_test,y_train,y_test=train_test_split(x_train_scaled,y_smote,test_size=0.2,random_state=42)


print(y)

0        1
1        2
2        3
3        1
4        3
        ..
52917    2
52918    2
52919    1
52920    2
52921    1
Length: 52922, dtype: int64


In [71]:
models={
    'LogisticRegression': LogisticRegression(max_iter=100),
    'RandomForestClassifier':RandomForestClassifier(n_estimators=100,max_depth=5,random_state=42),
    'DecisionTreeClassifier':DecisionTreeClassifier(max_depth=5,random_state=42),
    'XGBClassifier':XGBClassifier(n_estimator=100,max_depth=5,learning_rate=0.2,random_state=42)
}

for name,model in models.items():
    model.fit(x_train,y_train)
    y_pred=model.predict(x_test)
    ad=classification_report(y_test,y_pred)

    acu=accuracy_score(y_test,y_pred)
    pcu=precision_score(y_test,y_pred,average='weighted')
    re=recall_score(y_test,y_pred,average='weighted')
    f1=f1_score(y_test,y_pred,average='weighted')


    print(f'\n{name}')
    print(f'accuracy_score:{acu:.3f}')
    print(f'precision_score:{pcu:.3f}')
    print(f'recall_score:{re:.3f}')
    print(f'f1_score:{f1:.3f}')
    print('classification_report',ad)




LogisticRegression
accuracy_score:0.360
precision_score:0.367
recall_score:0.360
f1_score:0.332
classification_report               precision    recall  f1-score   support

           0       0.42      0.67      0.52      4364
           1       0.31      0.51      0.39      4327
           2       0.55      0.26      0.35      4359
           3       0.26      0.08      0.12      4314
           4       0.28      0.28      0.28      4253

    accuracy                           0.36     21617
   macro avg       0.37      0.36      0.33     21617
weighted avg       0.37      0.36      0.33     21617


RandomForestClassifier
accuracy_score:0.369
precision_score:0.445
recall_score:0.369
f1_score:0.315
classification_report               precision    recall  f1-score   support

           0       0.40      0.81      0.53      4364
           1       0.31      0.50      0.38      4327
           2       0.55      0.27      0.36      4359
           3       0.66      0.01      0.02      431

C:\Users\DEEPADHARSHINI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\training.py:183: UserWarning: [14:38:14] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "n_estimator" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



XGBClassifier
accuracy_score:0.452
precision_score:0.438
recall_score:0.452
f1_score:0.431
classification_report               precision    recall  f1-score   support

           0       0.55      0.84      0.66      4364
           1       0.36      0.42      0.39      4327
           2       0.49      0.44      0.46      4359
           3       0.39      0.18      0.25      4314
           4       0.41      0.37      0.39      4253

    accuracy                           0.45     21617
   macro avg       0.44      0.45      0.43     21617
weighted avg       0.44      0.45      0.43     21617



In [72]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Step 1: Train base model
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
xgb_model.fit(x_train, y_train)

# Step 2: Evaluate base model
y_pred = xgb_model.predict(x_test)
print("Base Model Performance:")
print(f'Accuracy: {accuracy_score(y_test, y_pred):.3f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"):.3f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"):.3f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"):.3f}')

# Step 3: Hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
}
xgb = xgb.XGBClassifier(random_state=42)
random_search = RandomizedSearchCV(
    xgb,
    param_distributions=param_grid,
    n_iter=5,
    cv=3,
    scoring='accuracy',  # ✅ Correct scoring for classification
    n_jobs=-1
)
random_search.fit(x_train, y_train)

# Step 4: Evaluate best model
best_xgb = random_search.best_estimator_
y_pred = best_xgb.predict(x_test)

print("\nTuned Model Performance:")
print(f'Accuracy: {accuracy_score(y_test, y_pred):.3f}')
print(f'Precision: {precision_score(y_test, y_pred, average="weighted"):.3f}')
print(f'Recall: {recall_score(y_test, y_pred, average="weighted"):.3f}')
print(f'F1 Score: {f1_score(y_test, y_pred, average="weighted"):.3f}')


Base Model Performance:
Accuracy: 0.457
Precision: 0.445
Recall: 0.457
F1 Score: 0.434

Tuned Model Performance:
Accuracy: 0.565
Precision: 0.552
Recall: 0.565
F1 Score: 0.556


In [75]:
import joblib
joblib.dump(best_xgb,'visit_xgb.pkl')
joblib.dump(ohe,'visit_ohe.pkl')
joblib.dump(te,'visit_te.pkl')
joblib.dump(label_encoder,'visit_label.pkl')
joblib.dump(sm,'visit_smote.pkl')
joblib.dump(scaler,'visit_scaler.pkl')

['visit_scaler.pkl']